In [1]:
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from shapely.geometry import Point
import geopandas as gpd
import numpy as np

### Load data

In [2]:
moves = dd.read_parquet('../data/moves_long.parquet', columns=['pid', 'from_addrid', 'to_addrid', 'to_effdate'])

In [5]:
properties = dd.read_csv(
    '../data/bay_area_properties*',
    dtype={'PROP_CENSUSTRACT': str, 'PROP_FIPSCD': str},
    usecols=['ADDRID', 'ADDRID2', 'DPV', 'PROP_LATITUDE', 'PROP_LONGITUDE', 'PROP_FIPSCD',
             'PROP_CENSUSTRACT', 'PROP_MUNINAME', 'PROP_OWNEROCC', 'PROP_QLTY',
             'PROP_VALCALC', 'PROP_UNVBLDSQFT', 'PROP_BEDRMS']
)

In [6]:
properties.head()

,DPV,PROP_FIPSCD,PROP_CENSUSTRACT,PROP_MUNINAME,PROP_LATITUDE,PROP_LONGITUDE,PROP_OWNEROCC,PROP_QLTY,PROP_VALCALC,PROP_UNVBLDSQFT,PROP_BEDRMS,ADDRID,ADDRID2
0,Y,06095,2529141010,VACAVILLE,38.3308,-121.934,O,QGO,440455.0,3104.0,5.0,102100585,102100585
1,Y,06095,2529141012,VACAVILLE,38.3331,-121.938,O,QGO,315865.0,2080.0,3.0,63630063,63630063
2,Y,06095,2529141013,VACAVILLE,38.3320,-121.936,O,QGO,608000.0,4128.0,5.0,236804971,236804971
3,Y,06095,2529152027,VACAVILLE,38.3255,-121.957,O,QAV,219974.0,1147.0,3.0,151925816,151925816
4,Y,06095,2529143001,VACAVILLE,38.3302,-121.959,O,QAV,181609.0,1110.0,3.0,198948195,198948195


### Filter data

In [7]:
prop_w_geog = properties[
    properties['PROP_LATITUDE'].notnull() & properties['PROP_LONGITUDE'].notnull() & 
    properties['DPV'].isin(['D', 'S', 'Y']) &
    properties['PROP_FIPSCD'].isin(['06001', '06013', '06041', '06055', '06075', '06081', '06085', '06095', '06097'])
]

### Merge on from ADDRID

In [8]:
moves = moves.set_index('from_addrid')

In [9]:
prop_w_geog = prop_w_geog.set_index('ADDRID')

In [10]:
prop_w_geog.head()

,DPV,PROP_FIPSCD,PROP_CENSUSTRACT,PROP_MUNINAME,PROP_LATITUDE,PROP_LONGITUDE,PROP_OWNEROCC,PROP_QLTY,PROP_VALCALC,PROP_UNVBLDSQFT,PROP_BEDRMS,ADDRID2
ADDRID,,,,,,,,,,,,
1838,Y,06081,6057002008,NaN,37.540800,-122.34700,O,NaN,3927917.0,5670.0,5.0,1838
1840,Y,06081,6009001019,NaN,37.704800,-122.48000,A,NaN,93198.0,1330.0,2.0,20814887
1843,Y,06075,0113002003,NaN,37.793109,-122.40902,O,NaN,1159096.0,1225.0,2.0,1843
1845,Y,06075,0131012005,NaN,37.791416,-122.42425,O,QEX,776701.0,1500.0,2.0,1845
1846,Y,06075,0214002003,NaN,37.749214,-122.42833,O,NaN,3774000.0,1250.0,NaN,1846


In [11]:
merged_1 = moves.merge(prop_w_geog, left_index=True, right_index=True)

In [12]:
with ProgressBar():
    merged_1 = merged_1.rename(columns={'PROP_LATITUDE': 'from_lat', 'PROP_LONGITUDE': 'from_lon'}).compute()

[########################################] | 100% Completed | 39.4s


In [14]:
moves_w_from_geog = merged_1.reset_index().rename(columns={'index': 'from_addrid'})

In [15]:
moves_w_from_geog.head()

,from_addrid,pid,to_addrid,to_effdate,DPV,PROP_FIPSCD,PROP_CENSUSTRACT,PROP_MUNINAME,from_lat,from_lon,PROP_OWNEROCC,PROP_QLTY,PROP_VALCALC,PROP_UNVBLDSQFT,PROP_BEDRMS,ADDRID2
0,1838.0,Y39394800924986,201439665.0,200509.0,Y,06081,6057002008,NaN,37.540800,-122.34700,O,NaN,3927917.0,5670.0,5.0,1838
1,1840.0,Y39394830916146,84669265.0,200002.0,Y,06081,6009001019,NaN,37.704800,-122.48000,A,NaN,93198.0,1330.0,2.0,20814887
2,1840.0,Y39394431814516,20814887.0,199912.0,Y,06081,6009001019,NaN,37.704800,-122.48000,A,NaN,93198.0,1330.0,2.0,20814887
3,1843.0,Y39394615847176,56523799.0,200004.0,Y,06075,0113002003,NaN,37.793109,-122.40902,O,NaN,1159096.0,1225.0,2.0,1843
4,1845.0,Y39394668254684,71161461.0,198503.0,Y,06075,0131012005,NaN,37.791416,-122.42425,O,QEX,776701.0,1500.0,2.0,1845


In [16]:
len(moves_w_from_geog)

7568917

### Merge on to ADDRID

In [17]:
moves_w_from_geog = moves_w_from_geog.set_index('to_addrid')

In [18]:
merged_2 = prop_w_geog.merge(moves_w_from_geog, left_index=True, right_index=True, suffixes=('_to', '_from'))

In [19]:
with ProgressBar():
    merged_2 = merged_2.rename(columns={'PROP_LATITUDE': 'to_lat', 'PROP_LONGITUDE': 'to_lon'}).compute()

[########################################] | 100% Completed | 43.0s


In [20]:
moves_w_geog = merged_2.reset_index().rename(columns={'index': 'to_addrid'})

In [21]:
moves_w_geog.head()

,to_addrid,DPV_to,PROP_FIPSCD_to,PROP_CENSUSTRACT_to,PROP_MUNINAME_to,to_lat,to_lon,PROP_OWNEROCC_to,PROP_QLTY_to,PROP_VALCALC_to,...,PROP_CENSUSTRACT_from,PROP_MUNINAME_from,from_lat,from_lon,PROP_OWNEROCC_from,PROP_QLTY_from,PROP_VALCALC_from,PROP_UNVBLDSQFT_from,PROP_BEDRMS_from,ADDRID2_from
0,1838,Y,06081,6057002008,NaN,37.540800,-122.34700,O,NaN,3927917.0,...,6028003002,NaN,37.655700,-122.48500,O,NaN,439898.0,2790.0,3.0,201439665
1,1838,Y,06081,6057002008,NaN,37.540800,-122.34700,O,NaN,3927917.0,...,6057004012,NaN,37.556300,-122.35300,O,NaN,4767552.0,5290.0,4.0,70211557
2,1840,Y,06081,6009001019,NaN,37.704800,-122.48000,A,NaN,93198.0,...,5113021024,PALO ALTO CITY,37.445200,-122.16500,A,QAV,409681.0,1307.0,4.0,138608104
3,1850,Y,06075,0612002003,NaN,37.736776,-122.39245,A,NaN,49166.0,...,0263021012,NaN,37.711452,-122.43683,O,NaN,329765.0,1200.0,NaN,1329018
4,1851,Y,06075,0216001001,NaN,37.747154,-122.43502,A,QAV,407914.0,...,0167004005,NaN,37.772162,-122.43744,S,NaN,1109192.0,1328.0,3.0,112947439


In [22]:
moves_w_geog.columns

Index(['to_addrid', 'DPV_to', 'PROP_FIPSCD_to', 'PROP_CENSUSTRACT_to',
       'PROP_MUNINAME_to', 'to_lat', 'to_lon', 'PROP_OWNEROCC_to',
       'PROP_QLTY_to', 'PROP_VALCALC_to', 'PROP_UNVBLDSQFT_to',
       'PROP_BEDRMS_to', 'ADDRID2_to', 'from_addrid', 'pid', 'to_effdate',
       'DPV_from', 'PROP_FIPSCD_from', 'PROP_CENSUSTRACT_from',
       'PROP_MUNINAME_from', 'from_lat', 'from_lon', 'PROP_OWNEROCC_from',
       'PROP_QLTY_from', 'PROP_VALCALC_from', 'PROP_UNVBLDSQFT_from',
       'PROP_BEDRMS_from', 'ADDRID2_from'],
      dtype='object')

### Drop duplicates

In [23]:
uniq_moves_w_geog = moves_w_geog.drop_duplicates(['pid', 'from_addrid', 'to_addrid'])

In [24]:
num_total_moves = len(moves)
num_matched_moves = len(uniq_moves_w_geog)
print(
    'Matched both properties to property records with geographies '
    'for {0} of {1} ({2}%) of relocation records.'.format(
        str(num_matched_moves), str(num_total_moves), str(round(100 * num_matched_moves / num_total_moves, 1))))

Matched both properties to property records with geographies for 4610235 of 14768635 (31.2%) of relocation records.


### Create geog columns

In [25]:
uniq_moves_w_geog2 = uniq_moves_w_geog[[
    'pid', 'from_addrid', 'from_lat', 'from_lon', 'PROP_FIPSCD_from', 'PROP_CENSUSTRACT_from', 'PROP_MUNINAME_from',
    'PROP_OWNEROCC_from', 'PROP_QLTY_from', 'PROP_VALCALC_from', 'PROP_UNVBLDSQFT_from', 'PROP_BEDRMS_from',
    'to_addrid', 'to_lat', 'to_lon', 'PROP_FIPSCD_to', 'PROP_CENSUSTRACT_to', 'PROP_MUNINAME_to', 'PROP_OWNEROCC_to',
    'PROP_QLTY_to', 'PROP_VALCALC_to', 'PROP_UNVBLDSQFT_to', 'PROP_BEDRMS_to', 'to_effdate'
]]

In [26]:
def get_dist(df):
    
    moves_w_geog = df.copy()
    moves_w_geog['from_coords'] = list(zip(moves_w_geog.from_lon, moves_w_geog.from_lat))
    moves_w_geog['to_coords'] = list(zip(moves_w_geog.to_lon, moves_w_geog.to_lat))
    moves_w_geog['from_coords'] = moves_w_geog['from_coords'].apply(Point)
    moves_w_geog['to_coords'] = moves_w_geog['to_coords'].apply(Point)
    from_gs = gpd.GeoSeries(moves_w_geog['from_coords'], crs={'init' :'epsg:4326'}).to_crs(epsg='2768')
    to_gs = gpd.GeoSeries(moves_w_geog['to_coords'], crs={'init' :'epsg:4326'}).to_crs(epsg='2768')
    dists = from_gs.distance(to_gs)
    moves_w_geog['distance'] = dists

    return moves_w_geog[np.append(df.columns, 'distance')]

In [27]:
%%time
ddf = dd.from_pandas(
    uniq_moves_w_geog2,
    npartitions=10000)

CPU times: user 14.8 s, sys: 832 ms, total: 15.7 s
Wall time: 15.6 s


In [28]:
ddf.head()

,pid,from_addrid,from_lat,from_lon,PROP_FIPSCD_from,PROP_CENSUSTRACT_from,PROP_MUNINAME_from,PROP_OWNEROCC_from,PROP_QLTY_from,PROP_VALCALC_from,...,to_lon,PROP_FIPSCD_to,PROP_CENSUSTRACT_to,PROP_MUNINAME_to,PROP_OWNEROCC_to,PROP_QLTY_to,PROP_VALCALC_to,PROP_UNVBLDSQFT_to,PROP_BEDRMS_to,to_effdate
0,Y39394800924986,201439665.0,37.655700,-122.48500,06081,6028003002,NaN,O,NaN,439898.0,...,-122.34700,06081,6057002008,NaN,O,NaN,3927917.0,5670.0,5.0,201003.0
1,Y39394867335546,70211557.0,37.556300,-122.35300,06081,6057004012,NaN,O,NaN,4767552.0,...,-122.34700,06081,6057002008,NaN,O,NaN,3927917.0,5670.0,5.0,200002.0
2,Y39394610202108,145654720.0,37.445200,-122.16500,06085,5113021024,PALO ALTO CITY,A,QAV,409681.0,...,-122.48000,06081,6009001019,NaN,A,NaN,93198.0,1330.0,2.0,201003.0
3,Y39398442015961,76133946.0,37.711452,-122.43683,06075,0263021012,NaN,O,NaN,329765.0,...,-122.39245,06075,0612002003,NaN,A,NaN,49166.0,2660.0,NaN,201504.0
4,Y39394471595510,112947439.0,37.772162,-122.43744,06075,0167004005,NaN,S,NaN,1109192.0,...,-122.43502,06075,0216001001,NaN,A,QAV,407914.0,1145.0,2.0,200912.0


In [29]:
dtypes = uniq_moves_w_geog2.dtypes.apply(lambda x: x.name).to_dict()
dtypes.update({'distance': 'float'})

In [30]:
ddf = ddf.map_partitions(get_dist, meta=dtypes)

In [ ]:
with ProgressBar():
    moves_w_dists = ddf.compute()

[####                                    ] | 10% Completed | 14min  3.7s

In [35]:
moves_w_dists.head()

,pid,from_addrid,from_lat,from_lon,PROP_FIPSCD_from,PROP_CENSUSTRACT_from,PROP_MUNINAME_from,PROP_OWNEROCC_from,PROP_QLTY_from,PROP_VALCALC_from,...,PROP_FIPSCD_to,PROP_CENSUSTRACT_to,PROP_MUNINAME_to,PROP_OWNEROCC_to,PROP_QLTY_to,PROP_VALCALC_to,PROP_UNVBLDSQFT_to,PROP_BEDRMS_to,to_effdate,distance
0,Y39394800924986,201439665.0,37.655700,-122.48500,06081,6028003002,NaN,O,NaN,439898.0,...,06081,6057002008,NaN,O,NaN,3927917.0,5670.0,5.0,201003.0,17638.119062
1,Y39394867335546,70211557.0,37.556300,-122.35300,06081,6057004012,NaN,O,NaN,4767552.0,...,06081,6057002008,NaN,O,NaN,3927917.0,5670.0,5.0,200002.0,1800.050142
2,Y39394610202108,145654720.0,37.445200,-122.16500,06085,5113021024,PALO ALTO CITY,A,QAV,409681.0,...,06081,6009001019,NaN,A,NaN,93198.0,1330.0,2.0,201003.0,40053.036820
3,Y39398442015961,76133946.0,37.711452,-122.43683,06075,0263021012,NaN,O,NaN,329765.0,...,06075,0612002003,NaN,A,NaN,49166.0,2660.0,NaN,201504.0,4817.170962
4,Y39394471595510,112947439.0,37.772162,-122.43744,06075,0167004005,NaN,S,NaN,1109192.0,...,06075,0216001001,NaN,A,QAV,407914.0,1145.0,2.0,200912.0,2783.668610


In [36]:
moves_w_dists.shape

(4610235, 25)

In [37]:
moves_w_dists.to_csv('../data/moves_w_dists.csv', index=False)